<a href="https://colab.research.google.com/github/tololojo/AML_FinalProject/blob/master/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import cv2
import tensorflow as tf
import torch
import math
import os
from skimage.metrics import structural_similarity as ssim
from skimage.color import deltaE_ciede2000
from torchvision.transforms import ToTensor
from torchvision import transforms

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Define the RGB to LAB and LAB to RGB conversion functions
def RGB2LAB2(R0, G0, B0):
    R = R0 / 255
    G = G0 / 255
    B = B0 / 255

    Y = 0.299 * R + 0.587 * G + 0.114 * B
    X = 0.449 * R + 0.353 * G + 0.198 * B
    Z = 0.012 * R + 0.089 * G + 0.899 * B

    L = Y
    a = (X - Y) / 0.234
    b = (Y - Z) / 0.785

    return L, a, b

def LAB22RGB(L, a, b):
    a11 = 0.299
    a12 = 0.587
    a13 = 0.114
    a21 = (0.15 / 0.234)
    a22 = (-0.234 / 0.234)
    a23 = (0.084 / 0.234)
    a31 = (0.287 / 0.785)
    a32 = (0.498 / 0.785)
    a33 = (-0.785 / 0.785)

    aa = np.array([[a11, a12, a13], [a21, a22, a23], [a31, a32, a33]])
    C0 = np.zeros((L.shape[0], 3))
    C0[:, 0] = L[:, 0]
    C0[:, 1] = a[:, 0]
    C0[:, 2] = b[:, 0]
    C = np.transpose(C0)

    X = np.linalg.inv(aa).dot(C)
    X1D = np.reshape(X, (X.shape[0] * X.shape[1], 1))
    p0 = np.where(X1D < 0)
    X1D[p0[0]] = 0
    p1 = np.where(X1D > 1)
    X1D[p1[0]] = 1
    Xr = np.reshape(X1D, (X.shape[0], X.shape[1]))

    Rr = Xr[0][:]
    Gr = Xr[1][:]
    Br = Xr[2][:]

    R = np.uint8(np.round(Rr * 255))
    G = np.uint8(np.round(Gr * 255))
    B = np.uint8(np.round(Br * 255))

    return R, G, B

# Define the transformation classes
class ConvertRGBToFeedModel(torch.nn.Module):
    def forward(self, img):
        img = np.asarray(img)
        sz0 = img.shape[0]
        sz1 = img.shape[1]

        train_imgs = np.zeros((sz0, sz1, 2))
        train_input = np.zeros((sz0, sz1, 1))

        ab = np.zeros((sz0, sz1, 2))
        R1 = np.reshape(img[:, :, 0], (sz0 * sz1, 1))
        G1 = np.reshape(img[:, :, 1], (sz0 * sz1, 1))
        B1 = np.reshape(img[:, :, 2], (sz0 * sz1, 1))
        L, A, B = RGB2LAB2(R1, G1, B1)
        A = np.reshape(A, (sz0, sz1))
        B = np.reshape(B, (sz0, sz1))
        ab[:, :, 0] = A
        ab[:, :, 1] = B
        train_input[:, :, :] = np.reshape(L, (sz0, sz1, 1))
        train_imgs[:, :, :] = np.reshape(ab, (sz0, sz1, 2))
        return train_input, train_imgs

class ConvertFromLABToRGB(torch.nn.Module):
    def forward(self, img_raw):
        image, _ = img_raw
        sz0 = image.shape[1]
        sz1 = image.shape[2]
        LAB_img = np.reshape(image, (sz0 * sz1, 2))
        L = LAB_img[:, 0:1]
        a = LAB_img[:, 1:2]

        R, G, B = LAB22RGB(L, a, a)
        R = np.reshape(R, (sz0, sz1))
        G = np.reshape(G, (sz0, sz1))
        B = np.reshape(B, (sz0, sz1))
        predicted255 = np.uint8(np.zeros((sz0, sz1, 3)))
        predicted255[:, :, 0] = R
        predicted255[:, :, 1] = G
        predicted255[:, :, 2] = B
        return predicted255

In [3]:
# Define the metric functions
def mae(imageA, imageB, bands):
    err = np.sum(np.abs(imageA.astype("float") - imageB.astype("float")))
    err /= float(imageA.shape[0] * imageA.shape[1] * bands)
    return err

def psnr(img1, img2):
    mse = np.mean((img1.astype("float") - img2.astype("float")) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def mse(imageA, imageB, nband):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1] * nband)
    return err

def rmse(imageA, imageB, nband):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1] * nband)
    err = np.sqrt(err)
    return err

In [ ]:
# Define the evaluation function
def evaluate_model(model, model_type, filesTs_list, files_name1, bands, N, save_dir):
    MSE_list = []
    MAE_list = []
    MSEr_list = []
    MSEg_list = []
    MSEb_list = []
    RMSE_list = []
    PSNR_list = []
    SSIM_list = []
    deltaE_list = []

    for j1 in range(N):
        print('Evaluating image = ', j1)
        ima = cv2.imread(filesTs_list[j1])
        ima0 = ima

        sz0 = ima.shape[0]
        sz1 = ima.shape[1]
        sz2 = bands

        ab = np.zeros((sz0, sz1, 2))
        R1 = np.reshape(ima0[:, :, 0], (sz0 * sz1, 1))
        G1 = np.reshape(ima0[:, :, 1], (sz0 * sz1, 1))
        B1 = np.reshape(ima0[:, :, 2], (sz0 * sz1, 1))
        L, A, B = RGB2LAB2(R1, G1, B1)
        A = np.reshape(A, (sz0, sz1))
        B = np.reshape(B, (sz0, sz1))
        ab[:, :, 0] = A
        ab[:, :, 1] = B
        ima_gray = np.reshape(L, (1, sz0, sz1, 1))

        if model_type == 'tensorflow':
            predicted = model.predict(ima_gray, verbose=0)
        elif model_type == 'pytorch':
            model.eval()
            with torch.no_grad():
                ima_gray_tensor = torch.from_numpy(ima_gray).float().unsqueeze(0)
                predicted_tensor = model(ima_gray_tensor)
                predicted = predicted_tensor.squeeze(0).numpy()

        predicted = np.reshape(predicted, (sz0 * sz1, bands))
        Ar = predicted[:, 0:1]
        Br = predicted[:, 1:2]

        Rr, Gr, Br = LAB22RGB(L, Ar, Br)
        Rr = np.reshape(Rr, (sz0, sz1))
        Gr = np.reshape(Gr, (sz0, sz1))
        Br = np.reshape(Br, (sz0, sz1))
        predicted255 = np.uint8(np.zeros((sz0, sz1, 3)))
        predicted255[:, :, 0] = Rr
        predicted255[:, :, 1] = Gr
        predicted255[:, :, 2] = Br

        # Save the generated image
        save_generated_image(predicted255, files_name1[j1], save_dir)

        MSE = mse(ima0, predicted255, 3)
        MSEr = mse(ima0[:, :, 0], predicted255[:, :, 0], 1)
        MSEg = mse(ima0[:, :, 1], predicted255[:, :, 1], 1)
        MSEb = mse(ima0[:, :, 2], predicted255[:, :, 2], 1)
        MAE = mae(ima0, predicted255, 3)
        RMSE = rmse(ima0, predicted255, 3)
        PSNR = psnr(ima0, predicted255)
        SSIM = ssim(ima0, predicted255, multichannel=True)
        deltaE_avg = np.mean(deltaE_ciede2000(L, Ar))

        MSE_list.append(MSE)
        MAE_list.append(MAE)
        MSEr_list.append(MSEr)
        MSEg_list.append(MSEg)
        MSEb_list.append(MSEb)
        RMSE_list.append(RMSE)
        PSNR_list.append(PSNR)
        SSIM_list.append(SSIM)
        deltaE_list.append(deltaE_avg)

    MSE_mean = np.mean(MSE_list)
    MAE_mean = np.mean(MAE_list)
    MSEr_mean = np.mean(MSEr_list)
    MSEg_mean = np.mean(MSEg_list)
    MSEb_mean = np.mean(MSEb_list)
    RMSE_mean = np.mean(RMSE_list)
    PSNR_mean = np.mean(PSNR_list)
    SSIM_mean = np.mean(SSIM_list)
    deltaE_mean = np.mean(deltaE_list)

    print("Evaluation Results:")
    print(f"MSE Mean: {MSE_mean}")
    print(f"MAE Mean: {MAE_mean}")
    print(f"MSEr Mean: {MSEr_mean}")
    print(f"MSEg Mean: {MSEg_mean}")
    print(f"MSEb Mean: {MSEb_mean}")
    print(f"RMSE Mean: {RMSE_mean}")
    print(f"PSNR Mean: {PSNR_mean}")
    print(f"SSIM Mean: {SSIM_mean}")
    print(f"DeltaE Mean: {deltaE_mean}")


In [16]:
# Define a function to save the generated images
def save_generated_image(image, filename, save_dir, file_extension='tif'):
    """
    Save the generated image to the specified directory.

    Args:
    - image: The generated image.
    - filename: The name of the file.
    - save_dir: The directory where the image will be saved.
    - file_extension: The file extension of the image (default: 'tif').
    """
    # Construct the full path for saving the image
    save_path = os.path.join(save_dir, f"{filename}.{file_extension}")
    # Save the image
    cv2.imwrite(save_path, image)


In [17]:
# EDefine the paths
data_path = '/content/drive/MyDrive/CAS Advanced Machine Learning/Luftbild_Colorization/Evaluation_Images'
save_dir = '/content/drive/MyDrive/CAS Advanced Machine Learning/Luftbild_Colorization/Evaluation_Results'
file_extension = 'jpeg'  # Change this to the desired file extension

In [18]:
# Get the list of files in the test images directory
filesTs_list = [os.path.join(data_path, file) for file in os.listdir(data_path)]
files_name1 = [os.path.splitext(os.path.basename(file))[0] for file in filesTs_list]
bands = 3
# Number of test images
N = len(filesTs_list)
print(N)

72


In [14]:
files_name1

['image_71_lat_46.42641854173257_lon_8.11771951517454_zoom_18_class_66',
 'image_69_lat_47.146173793218885_lon_7.591563788353546_zoom_17_class_52',
 'image_70_lat_46.488934064375385_lon_7.833269697917848_zoom_17_class_72',
 'image_44_lat_46.058235729742925_lon_7.999434279555997_zoom_17_class_62',
 'image_40_lat_46.313418953576075_lon_7.860558379522397_zoom_18_class_20',
 'image_42_lat_47.109908681704304_lon_9.281995061874811_zoom_18_class_16',
 'image_41_lat_46.03253915279717_lon_7.591047201409353_zoom_18_class_69',
 'image_43_lat_46.64989247422142_lon_9.403495251144596_zoom_17_class_45',
 'image_39_lat_46.461119565860315_lon_6.854157677314347_zoom_18_class_9',
 'image_29_lat_46.49679251748776_lon_7.370905677776521_zoom_16_class_11',
 'image_12_lat_46.473404345753906_lon_6.426936994655842_zoom_17_class_19',
 'image_13_lat_47.04705624800896_lon_7.958440499076391_zoom_16_class_51',
 'image_11_lat_46.16221238086289_lon_8.861479418973186_zoom_18_class_67',
 'image_27_lat_47.52452191086445_

In [ ]:
# Loop over the images and save them
for j1 in range(N):
    save_generated_image(predicted255, files_name1[j1], save_dir, file_extension)

In [ ]:
# Load your models
tensorflow_model = tf.keras.models.load_model('path/to/tensorflow_model.h5')
pytorch_model = torch.load('path/to/pytorch_model.pth')


In [ ]:
# Evaluate the TensorFlow model
print("Evaluating TensorFlow model...")
evaluate_model(tensorflow_model, 'tensorflow', filesTs_list, files_name1, bands, N, cwd)
